In [18]:
import json, os, parser

#    with open(file_name,'r') as file, open('/Users/ch/Documents/entity_relation_dict/relation_entity_dict_cate_diff/' + os.path.basename(file_name),'r') as fl:
#        fl.seek(0,0)
#        e1_e2_cate_diff = json.loads(fl.readline())


def read_entitydict_as_the_standard_file(standard_profile):
    entitydict_standard_dict = {}
    with open(standard_profile,'r') as file:
        file.seek(0,0)
        for line in file:
            entity = line.strip().split("\t")[0]
            cate = line.strip().split("\t")[1]
            entitydict_standard_dict[entity] = cate
    return entitydict_standard_dict

def deal_rela_entity_dict_diff(file_folder_path, entitydict_standard_dict):
    file_path_list = []
    for file_path_root,_,file_name_list in os.walk(file_folder_path):
        for file_name in file_name_list:
            if ".dict" in file_name:
                file_path_list.append(os.path.join(file_path_root,file_name))
    fc = open('./change_medical_entity_dict_v3.2.txt','a')
#    all_relation_list = ["drugs_drugs_interact","drugs_gene_target","drugs_protein_target","compound_drugs_contain","drugs_disease_treat","drugs_symptom_treat", \
 #                        "drugs_organism_apply","drugs_anatomy_affect","drugs_equipment_relation","drugs_phenomenon_occur","gene_gene_interact",\
  #                       "gene_protein_express","compound_gene_treat","gene_disease_cause","gene_symptom_relate","gene_organism_belong","gene_anatomy_locate",\
   #                      "gene_phenomenon_relate","protein_protein_interact","compound_protein_treat","protein_disease_cause","protein_symptom_1realte2cause", \
    #                     "protein_organism_belong","protein_anatomy_belong","protein_phenomenon_relate","compound_disease_cause","compound_symptom_cause", \
     #                    "compound_phenomenon_cause","disease_disease_coexist","disease_symptom_coexist","disease_anatomy_occur","equipment_disease_diagnose/treat", \
      #                   "disease_phenomenon_occur","symptom_symptom_coexist","symptom_organism_occur","symptom_anatomy_occur","equipment_symptom_diagnose/treat", \
       #                  "organism_organism_coexist","anatomy_organism_belong","phenomenon_organism_occur","equipment_equipment_relate","phenomenon_phenomenon_relate"]

    cate_diff_list = set()
    del_relation_dict_count = 0
    num = 0
    for file_name in file_path_list:
        num = num + 1
        print(num)
        try:
            with open(file_name,'r') as file, open('./Data/SummaryChihuiDifferentCategoryDict_v3.2/' + os.path.basename(file_name),'r') as fl,\
                open('./relation_entity_dict_align_v3.1/' + os.path.basename(file_name), 'w') as fw:
                fl.seek(0,0)
                e1_e2_cate_diff = json.loads(fl.readline())
    #            change_relation(file_name)
                entitydict_as_the_standard_dict = entitydict_standard_dict
                change_relation_dict = ["drugs_compound"]
                change_medical_entity_dict = ["compound_drugs","gene_protein","anatomy_disease","anatomy_protein","compound_equipment","compound_phenomenon"]
                del_relation_dict = ["compound_disease","compound_protein","compound_gene","compound_anatomy","compound_organism", \
                                     "disease_compound","disease_equipment","disease_protein","disease_symptom","disease_phenomenon","disease_anatomy","disease_organism","disease_drugs","disease_gene", \
                                     "drugs_disease","drugs_protein","drugs_gene","drugs_organism","drugs_anatomy","drugs_equipment","drugs_phenomenon", \
                                     "gene_disease","gene_compound","gene_equipment","gene_anatomy","gene_drugs","gene_organism","gene_phenomenon","gene_symptom", \
                                     "protein_compound","protein_disease","protein_gene","protein_equipment","protein_drugs","protein_anatomy","protein_phenomenon","protein_organism","protein_symptom"]

                file.seek(0,0)
                for line in file:
                    line_dict = json.loads(line.strip())
                    e1 = line_dict["e1"]
                    if e1 in e1_e2_cate_diff:      #  如果关系字典中 e1 的 category 与实体字典中的不一样那么就获取实体字典中的category
                        e1_cate_diff = e1_e2_cate_diff[e1]
                        e1_entity_cate = e1_cate_diff.split("_")[1]   #   e1 在实体字典中的category
                        flag_e1 = 1
                    else:
                        e1_cate = os.path.basename(file_name).split("_")[0]
                        flag_e1 = 0

                    e2 = line_dict["e2"]
                    if e2 in e1_e2_cate_diff:
                        e2_cate_diff = e1_e2_cate_diff[e2]
                        e2_entity_cate = e2_cate_diff.split("_")[1]    #    e2 在实体字典中的category
                        flag_e2 = 1
                    else:
                        e2_cate = os.path.basename(file_name).split("_")[1]
                        flag_e2 = 0

    #                e1_e2_cate_combine = e1_cate + "_" + e2_cate
    #           如果 e1 和 e2 的 category 在关系字典和实体字典中均是相同的，那么就输出      
                    if flag_e1 == 0 and flag_e2 == 0:
                        fw.write(json.dumps(line_dict) + '\n')
                        continue
                    elif flag_e1 == 1 and flag_e2 == 0:
    #           如果 e1的 category类型在改变关系字典的dict中，那么就生成一个新的关系
    #                    if e1_cate_diff in change_relation_dict:
                        if (e1_cate_diff in change_relation_dict) or (e1 in entitydict_as_the_standard_dict):
                            with open('./change_relation_dict_v3.1/'+ e1_cate_diff.split("_")[1] + '_' + e2_cate + '.dict','a') as f:
                                f.write(json.dumps(line_dict) + '\n')
                                continue
    #           如果 e1的 category类型在改变实体字典的dict中，那么就将需要纠正的 name 和 category 存在一个文件中                            
                        elif (e1_cate_diff in change_medical_entity_dict) and (e1 not in entitydict_as_the_standard_dict):
                            fc.write(e1 + '\t' + e1_entity_cate + '\t' + os.path.basename(file_name).split("_")[0] + '\n')
    #                        fc.write(e2 + '\t' + os.path.basename(file_name).split("_")[1] + '\n')
                            fw.write(json.dumps(line_dict) + '\n')
                            continue
    #           如果 e1的 category类型在删除关系对的字典中，那么就将该关系对跳过，不处理也不输出
                        elif e1_cate_diff in del_relation_dict:
                            del_relation_dict_count += 1
                            continue
                        elif (e1_cate_diff not in change_relation_dict) and (e1_cate_diff not in change_medical_entity_dict) and (e1_cate_diff not in del_relation_dict):
                            cate_diff_list.append(e1_cate_diff)
    #                        fw.write(json.dumps(line_dict) + '\n')
                            continue
                    elif flag_e1 == 0 and flag_e2 == 1:
                        if (e2_cate_diff in change_relation_dict) or (e2 in entitydict_as_the_standard_dict):
                            with open('./change_relation_dict_v3.1/'+ e1_cate + '_' + e2_cate_diff.split("_")[1] + '.dict','a') as f:
                                f.write(json.dumps(line_dict) + '\n')
                                continue
                        elif (e2_cate_diff in change_medical_entity_dict) and (e2 not in entitydict_as_the_standard_dict):
                            fc.write(e2 + '\t' + e2_entity_cate + '\t' + os.path.basename(file_name).split("_")[1] + '\n')
                            fw.write(json.dumps(line_dict) + '\n')
                            continue
                        elif e2_cate_diff in del_relation_dict:
                            del_relation_dict_count += 1
                            continue
                        elif (e2_cate_diff not in change_relation_dict) and (e2_cate_diff not in change_medical_entity_dict) and (e2_cate_diff not in del_relation_dict):
                            cate_diff_list.append(e2_cate_diff)
    #                        fw.write(json.dumps(line_dict) + '\n')
                            continue
                    elif flag_e1 == 1 and flag_e2 == 1:
    #                    if (e1_cate_diff in change_relation_dict) and (e2_cate_diff in del_relation_dict):
    #                        continue
                        if (e1_cate_diff in del_relation_dict) or (e2_cate_diff in del_relation_dict):
                            del_relation_dict_count += 1
                            continue
                        elif ((e1_cate_diff in change_medical_entity_dict) and (e1 not in entitydict_as_the_standard_dict)) and ((e2_cate_diff in change_medical_entity_dict) and (e2 not in entitydict_as_the_standard_dict)):
                            fc.write(e1 + '\t' + e1_entity_cate + '\t' + os.path.basename(file_name).split("_")[0] + '\n')
                            fc.write(e2 + '\t' + e2_entity_cate + '\t' + os.path.basename(file_name).split("_")[1] + '\n')
                            fw.write(json.dumps(line_dict) + '\n')
                            continue
                        elif ((e1_cate_diff in change_relation_dict) or (e1 in entitydict_as_the_standard_dict)) and ((e2_cate_diff in change_medical_entity_dict) and (e2 not in entitydict_as_the_standard_dict)):
                            with open('./change_relation_dict_v3.1/'+ e1_cate_diff.split("_")[1] + '_' + e2_cate + '.dict','a') as f:
                                f.write(json.dumps(line_dict) + '\n')
                            fc.write(e2 + '\t' + e2_entity_cate + '\t' + os.path.basename(file_name).split("_")[1] + '\n')
                            continue
                        elif ((e1_cate_diff in change_medical_entity_dict) and (e1 not in entitydict_as_the_standard_dict)) and ((e2_cate_diff in change_relation_dict) or (e2 in entitydict_as_the_standard_dict)):
                            with open('./change_relation_dict_v3.1/'+ e1_cate + '_' + e2_cate_diff.split("_")[1] + '.dict','a') as f:
                                f.write(json.dumps(line_dict) + '\n')
                            fc.write(e1 + '\t' + e1_entity_cate + '\t' + os.path.basename(file_name).split("_")[0] + '\n')
                            continue
                        elif ((e1_cate_diff in change_relation_dict) or (e1 in entitydict_as_the_standard_dict)) and ((e2_cate_diff in change_relation_dict) or (e2 in entitydict_as_the_standard_dict)):
                            with open('./change_relation_dict_v3.1/'+ e1_cate_diff.split("_")[1] + '_' + e2_cate_diff.split("_")[1] + '.dict','a') as f:
                                f.write(json.dumps(line_dict) + '\n')
                                continue
                        else:
    #                        fw.write(json.dumps(line_dict) + '\n')
                            if (e1_cate_diff not in change_relation_dict) and (e1_cate_diff not in change_medical_entity_dict) and (e1_cate_diff not in del_relation_dict):
                                cate_diff_list.append(e1_cate_diff)
                            if (e2_cate_diff not in change_relation_dict) and (e2_cate_diff not in change_medical_entity_dict) and (e2_cate_diff not in del_relation_dict):
                                cate_diff_list.append(e2_cate_diff)
    #                    else:
    #                        continue

        
        except Exception as e:
            print(e)
    fc.close() 
if __name__ == "__main__":
#    file_folder_path = r"./relation_dict_self_align2/"
#    file_folder_path = r'./Data/NewEntityRelationDict_v2.1/'
    file_folder_path = r'./relation_entity_dict_align_v3.0/'
    standard_profile = r'./Data/entitydict_as_the_standard'
    entitydict_standard_dict = read_entitydict_as_the_standard_file(standard_profile)
    deal_rela_entity_dict_diff(file_folder_path, entitydict_standard_dict)
    

1
2
3
[Errno 2] No such file or directory: './Data/SummaryChihuiDifferentCategoryDict_v3.2/equipment_disease_diagnose|treat.dict'
4
[Errno 2] No such file or directory: './Data/SummaryChihuiDifferentCategoryDict_v3.2/drugs_anatomy_affect.dict'
5
6
7
8
[Errno 2] No such file or directory: './Data/SummaryChihuiDifferentCategoryDict_v3.2/drugs_organism_apply.dict'
9
[Errno 2] No such file or directory: './Data/SummaryChihuiDifferentCategoryDict_v3.2/disease_anatomy_occur.dict'
10
[Errno 2] No such file or directory: './Data/SummaryChihuiDifferentCategoryDict_v3.2/compound_drugs_contain.dict'
11
[Errno 2] No such file or directory: './Data/SummaryChihuiDifferentCategoryDict_v3.2/protein_anatomy_belong.dict'
12
13
14
[Errno 2] No such file or directory: './Data/SummaryChihuiDifferentCategoryDict_v3.2/drugs_disease_treat.dict'
15
[Errno 2] No such file or directory: './Data/SummaryChihuiDifferentCategoryDict_v3.2/disease_disease_coexist.dict'
16
[Errno 2] No such file or directory: './Data/S

In [29]:
import json
#  将两个库中的数据合在一个文件里  （例：将 malacards 和 drugbank 的数据和在一个文件里）
with open('./change_relation_dict_v3.1/protein_protein.dict', mode='r') as file,open('./relation_entity_dict_align_v3.1/protein_protein_interact.dict', 'a') as fl:
        
        file.seek(0,0)
        line = file.readline()
        
        while(line):
            line_dict = json.loads(line.strip())        
            fl.write(json.dumps(line_dict) + '\n')
            
            line = file.readline()
#         fl.close()

In [26]:
#  e1、e2实体互换
import json

with open('./change_relation_dict_v3.1/protein_gene.dict','r') as file, \
        open('./change_relation_dict_v3.1/gene_protein_express.dict','a') as fl:
    
    file.seek(0,0)
    for line in file:
        line_dict = json.loads(line.strip())
        e1 = line_dict["e2"]
        e2 = line_dict["e1"]
        
        line_dict["e1"] = e1
        line_dict["e2"] = e2
#        if "()" in line_dict["e2"]:
#        if (line_dict["e2"] == "chong-myung-tang") or (line_dict["e2"] == "dai-kenchu-to"):
#            continue
#        else:
        fl.write(json.dumps(line_dict) + "\n")
    

In [7]:
#  统计关系字典中自对齐问题

def readLinesFile(read_file_path):
    with open(read_file_path,"r",encoding="utf-8") as fp:
        lines=fp.readlines()
    return lines

def writeJsonFile(dictLineJson,write_json_file_path):
    with open(write_json_file_path,"a",encoding="utf-8") as fp_w_json:
        json.dump(dictLineJson,fp_w_json)  #write file
        fp_w_json.write("\n")

def statRelDictsSameEntityDifferentCategoryAmount(read_entity_relation_dict_dir ,write_stat_path ):
    # header = ["entity",""]
    stat_dict = {}
    rel_dict_file_list = os.listdir(read_entity_relation_dict_dir)
    stat_different_amount = 0

    for rel_dict_file_name in rel_dict_file_list:
        if rel_dict_file_name == ".DS_Store":
            continue
        rel_dict_path = os.path.join(read_entity_relation_dict_dir,rel_dict_file_name)
        lines = readLinesFile(rel_dict_path)
        c1 = rel_dict_file_name.split("_")[0]
        c2 = rel_dict_file_name.split("_")[1]
        two_key = rel_dict_file_name[:-5]
        for line in lines:
            line = json.loads(line)
            e1, e2 =line["e1"], line["e2"]
            source = line["sources"]
            # if c1 == c2 or e1 == e2:
            #     continue
            if e1 not in stat_dict:
                # stat_dict[e1] = {two_key: [c1]}
                stat_dict[e1] ={two_key :{c1:source}}
                stat_different_amount+=1
            else:
                if two_key not in stat_dict[e1]:
                    # stat_dict[e1][two_key] = [c1]
                    stat_dict[e1][two_key] = {c1:source}
                    stat_different_amount += 1

                if c1 not in stat_dict[e1][two_key]: # normal unexist!!1
                    # stat_dict[e1][two_key].append(c1)
                    stat_dict[e1][two_key][c1] =source
                    stat_different_amount += 1

            if e2 not in stat_dict:
                # stat_dict[e2] ={two_key :[c2]}
                stat_dict[e2] = {two_key: {c2:source}}
                stat_different_amount+=1

            else:
                if two_key not in stat_dict[e2]:
                    # stat_dict[e2][two_key] = [c2]
                    stat_dict[e2][two_key] = {c2:source}
                    stat_different_amount += 1

                if c2 not in stat_dict[e2][two_key]:
                    # stat_dict[e2][two_key].append(c2)
                    stat_dict[e2][two_key][c2] =source
                    stat_different_amount += 1

            # stat_dict[e1][rel_dict_file_name[:-5]] = c1
            # stat_dict[e2][rel_dict_file_name[:-5]] = c2
    for key,value in stat_dict.items():
        # value_v_list = list(value.values())
        value_v_list = []
        temp_set= set()
        for value_k,value_v in value.items():
            # value_v_list = list(value_v.keys())
            for value_v_k, value_v_v in value_v.items():
                temp_set.add(value_v_k)
        value_v_list = list(temp_set)
        value_v_list_len  = len(value_v_list)
        if value_v_list_len == 1:
            continue
        for i in range(1,value_v_list_len):
            if value_v_list[i-1] == value_v_list[1]:
                continue
            else:
                # print("write data...")
                writeJsonFile({key:value}, write_stat_path)
                break


    print("stat_different_amount = ",stat_different_amount)
    

if __name__ == "__main__":
    #todo: note: execute below function, the order is very important!!!
    #todo: statistics not align entity!!!, this is the first statistics!
    read_entity_relation_dict_dir = "./quchong_relation_dict/"
    write_stat_path ="./rel_dicts_same_entity_category_different_1.json"
    statRelDictsSameEntityDifferentCategoryAmount(read_entity_relation_dict_dir, write_stat_path)   
    
    

stat_different_amount =  393934


In [18]:
print(cate_diff_list)

set()


In [19]:
print(del_relation_dict_count)

8240790


In [1]:
all_relation_list = ["drugs_drugs_interact","drugs_gene_target","drugs_protein_target","compound_drugs_contain","drugs_disease_treat","drugs_symptom_treat", \
                    "drugs_organism_apply","drugs_anatomy_affect","drugs_equipment_relation","drugs_phenomenon_occur","gene_gene_interact",\
                    "gene_protein_express","compound_gene_treat","gene_disease_cause","gene_symptom_relate","gene_organism_belong","gene_anatomy_locate",\
                    "gene_phenomenon_relate","protein_protein_interact","compound_protein_treat","protein_disease_cause","protein_symptom_1realte2cause", \
                    "protein_organism_belong","protein_anatomy_belong","protein_phenomenon_relate","compound_disease_cause","compound_symptom_cause", \
                    "compound_phenomenon_cause","disease_disease_coexist","disease_symptom_coexist","disease_anatomy_occur","equipment_disease_diagnose/treat", \
                    "disease_phenomenon_occur","symptom_symptom_coexist","symptom_organism_occur","symptom_anatomy_occur","equipment_symptom_diagnose/treat", \
                    "organism_organism_coexist","anatomy_organism_belong","phenomenon_organism_occur","equipment_equipment_relate","phenomenon_phenomenon_relate"]
print(len(all_relation_list))

42
